In [1]:
import pydot
import tensorflow as tf
import string
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras_preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
# small library for seeing the progress of loops.
from tqdm.notebook import tqdm
tqdm.pandas()

In [3]:
import numpy as np
from nltk.translate.bleu_score import corpus_bleu
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from pickle import load
from PIL import Image

# Step 1: Load descriptions in dictionary format
def load_descriptions_to_dict(filename):
    descriptions = {}
    with open(filename, 'r') as file:
        for line in file:
            tokens = line.strip().split('\t')
            if len(tokens) < 2:
                continue
            image_id, caption = tokens[0], tokens[1]
            image_id = image_id.split('.')[0]  # Remove file extension
            if image_id not in descriptions:
                descriptions[image_id] = []
            descriptions[image_id].append(caption)
    return descriptions

def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo, sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

In [5]:
descriptions_file = 'descriptions.txt'  # Replace with your file path
descriptions = load_descriptions_to_dict(descriptions_file)
descriptions

{'1000268201_693b08cb0e': ['child in pink dress is climbing up set of stairs in an entry way',
  'girl going into wooden building',
  'little girl climbing into wooden playhouse',
  'little girl climbing the stairs to her playhouse',
  'little girl in pink dress going into wooden cabin'],
 '1001773457_577c3a7d70': ['black dog and spotted dog are fighting',
  'black dog and tricolored dog playing with each other on the road',
  'black dog and white dog with brown spots are staring at each other in the street',
  'two dogs of different breeds looking at each other on the road',
  'two dogs on pavement moving toward each other'],
 '1002674143_1b742ab4b8': ['little girl covered in paint sits in front of painted rainbow with her hands in bowl',
  'little girl is sitting in front of large painted rainbow',
  'small girl in the grass plays with fingerpaints in front of white canvas with rainbow on it',
  'there is girl with pigtails sitting in front of rainbow painting',
  'young girl with pi

In [6]:
# Step 3: Verify descriptions
print(f"Loaded {len(descriptions)} descriptions.")
example_id = list(descriptions.keys())[0]
print(f"Example captions for {example_id}: {descriptions[example_id]}")

Loaded 8092 descriptions.
Example captions for 1000268201_693b08cb0e: ['child in pink dress is climbing up set of stairs in an entry way', 'girl going into wooden building', 'little girl climbing into wooden playhouse', 'little girl climbing the stairs to her playhouse', 'little girl in pink dress going into wooden cabin']


In [7]:
# Step 5: Load model and tokenizer
model = load_model('models/model_10.keras')  # Replace with your model path
tokenizer = load(open("tokenizer.p", "rb"))

# Step 6: Load image features
features = load(open("features.p", "rb"))


In [8]:
features

{'1000268201_693b08cb0e.jpg': array([[0.47339684, 0.01732643, 0.07333998, ..., 0.0855904 , 0.02102295,
         0.23766544]], dtype=float32),
 '1001773457_577c3a7d70.jpg': array([[0.00158211, 0.11113487, 0.00037397, ..., 0.26503602, 0.35279822,
         0.05871649]], dtype=float32),
 '1002674143_1b742ab4b8.jpg': array([[0.        , 0.02488983, 0.01554059, ..., 0.        , 0.        ,
         0.10192626]], dtype=float32),
 '1003163366_44323f5815.jpg': array([[0.14568879, 0.00272414, 0.27776527, ..., 0.17018232, 0.11957303,
         0.09414067]], dtype=float32),
 '1007129816_e794419615.jpg': array([[0.        , 0.12443952, 0.7391621 , ..., 0.00390435, 0.00997149,
         0.5017237 ]], dtype=float32),
 '1007320043_627395c3d8.jpg': array([[0.04136695, 0.        , 0.0127447 , ..., 0.00944686, 0.64201635,
         0.04792112]], dtype=float32),
 '1009434119_febe49276a.jpg': array([[0.        , 0.        , 0.02624729, ..., 0.30528584, 0.23091821,
         0.14191067]], dtype=float32),
 '1012

In [14]:
print("Keys in descriptions:", list(descriptions.keys())[:])  # Print first 5 keys
print("Keys in features:", list(features.keys())[:])  # Print first 5 keys

Keys in descriptions: ['1000268201_693b08cb0e', '1001773457_577c3a7d70', '1002674143_1b742ab4b8', '1003163366_44323f5815', '1007129816_e794419615', '1007320043_627395c3d8', '1009434119_febe49276a', '1012212859_01547e3f17', '1015118661_980735411b', '1015584366_dfcec3c85a', '101654506_8eb26cfb60', '101669240_b2d3e7f17b', '1016887272_03199f49c4', '1019077836_6fc9b15408', '1019604187_d087bf9a5f', '1020651753_06077ec457', '1022454332_6af2c1449a', '1022454428_b6b660a67b', '1022975728_75515238d8', '102351840_323e3de834', '1024138940_f1fefbdce1', '102455176_5f8ead62d5', '1026685415_0431cbf574', '1028205764_7e8df9a2ea', '1030985833_b0902ea560', '103106960_e8a41d64f8', '103195344_5d2dc613a3', '103205630_682ca7285b', '1032122270_ea6f0beedb', '1032460886_4a598ed535', '1034276567_49bb87c51c', '104136873_5b5d41be75', '1042020065_fb3d3ba5ba', '1042590306_95dea0916c', '1045521051_108ebc19be', '1048710776_bb5b0a5c7c', '1052358063_eae6744153', '105342180_4d4a40b47f', '1053804096_ad278b25f1', '1055623002

In [10]:
# Remove extensions from keys in features
features = {key.split('.')[0]: value for key, value in features.items()}

# Verify the updated keys
print("Keys in features after removing extensions:", list(features.keys())[:5])


Keys in features after removing extensions: ['1000268201_693b08cb0e', '1001773457_577c3a7d70', '1002674143_1b742ab4b8', '1003163366_44323f5815', '1007129816_e794419615']


In [13]:
import numpy as np
from nltk.translate.bleu_score import corpus_bleu
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from pickle import load
from PIL import Image



# Step 4: Define evaluation function (reuse `evaluate_model` as before)
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()

    for key, desc_list in descriptions.items():
        # Generate prediction
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
        # Strip 'start' and 'end' tokens
        yhat = yhat.split()[1:-1]

        # Store actual and predicted captions
        references = [d.split() for d in desc_list]
        actual.append(references)
        predicted.append(yhat)

    # Calculate BLEU scores
    bleu1 = corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0))
    bleu2 = corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0))
    bleu3 = corpus_bleu(actual, predicted, weights=(0.33, 0.33, 0.33, 0))
    bleu4 = corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25))

    print(f'BLEU-1: {bleu1:.3f}')
    print(f'BLEU-2: {bleu2:.3f}')
    print(f'BLEU-3: {bleu3:.3f}')
    print(f'BLEU-4: {bleu4:.3f}')
  # Replace with your image features dictionary loading logic
max_length=32
# Step 7: Evaluate model
evaluate_model(model, descriptions, features, tokenizer, max_length)


KeyError: '2258277193_586949ec62'